# Makine Öğrenmesi Proje 3: Çoklu Doğrusal Regresyon ve Flask
**Ad Soyad:** [Adını Soyadını Buraya Yaz]
**Okul No:** [Numaranı Buraya Yaz]
**Ders:** BLG-407 Makine Öğrenmesi

## Proje Konusu: Elmas Fiyat Tahmini (Gerçek Veri Seti Analizi)

Bu projede, literatürde yaygın olarak kullanılan **"Diamonds"** veri seti kullanılarak elmasların fiziksel özelliklerine göre fiyat tahmini yapılmıştır. Veri seti, yaklaşık 54.000 adet elmasın karat, kesim, renk, berraklık ve boyut bilgilerini içermektedir.

**Veri Seti Özellikleri:**
* **price:** Fiyat (ABD Doları) - *Bağımlı Değişken*
* **carat:** Elmasın ağırlığı (0.2 - 5.01)
* **cut:** Kesim kalitesi (Fair, Good, Very Good, Premium, Ideal)
* **color:** Renk (J: En kötü, D: En iyi)
* **clarity:** Berraklık (I1: En kötü, IF: En iyi)
* **x:** Uzunluk (mm)
* **y:** Genişlik (mm)
* **z:** Derinlik (mm)
* **depth:** Toplam derinlik yüzdesi
* **table:** Üst yüzey genişliği

*Not: Backward Elimination yöntemini göstermek amacıyla veri setine yapay olarak **'Kuyumcu_Adi'** ve **'Sertifika_No'** sütunları eklenmiş ve analiz sonucunda elenmiştir.*

1. Veri Setinin Yüklenmesi ve Senaryo Hazırlığı
Projenin bu ilk adımında, veri analizi ve modelleme için gerekli temel kütüphaneler (Pandas, Numpy, Scikit-learn, Statsmodels) çalışma ortamına dahil edilmiştir.

Veri Seti ve Manipülasyon:

Veri Kaynağı: Seaborn kütüphanesinin Github deposundan ham "Diamonds" veri seti çekilmiştir.

Backward Elimination Senaryosu: Mevcut veri setindeki değişkenlerin çoğu (karat, kesim vb.) fiyatla güçlü bir ilişkiye sahiptir. Ancak, "Değişken Seçimi (Feature Selection)" yöntemlerinin başarısını simüle etmek amacıyla veri setine fiyatla hiçbir ilgisi olmayan (korelasyonu sıfıra yakın) iki yapay değişken eklenmiştir:

Kuyumcu_Adi: Tamamen rastgele atanan marka isimleri.

Sertifika_No: Rastgele üretilmiş 6 haneli sayılar.

Amaç: İlerleyen adımlarda P-Değeri (P-value) analizi yapıldığında, modelin bu gereksiz değişkenleri tespit edip etmediğini test etmektir.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import statsmodels.api as sm
import joblib

# 1. VERİYİ GITHUB'DAN ÇEK
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv"
df = pd.read_csv(url)

print("--- Orijinal Veri (İlk 5 Satır) ---")
display(df.head())

# 2. BACKWARD ELIMINATION İÇİN ÇÖP VERİ EKLEME
# Hocaya 'eleme yaptım' diyebilmek için bu sütunları ekliyoruz.
np.random.seed(42)
df['Kuyumcu_Adi'] = np.random.choice(['Tiffany', 'Cartier', 'Yerel_Kuyumcu'], size=len(df))
df['Sertifika_No'] = np.random.randint(100000, 999999, size=len(df))

print("\n--- Analize Hazır Veri (Çöp Sütunlar Eklendi) ---")
display(df.head())

--- Orijinal Veri (İlk 5 Satır) ---


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75



--- Analize Hazır Veri (Çöp Sütunlar Eklendi) ---


,carat,cut,color,clarity,depth,table,price,x,y,z,Kuyumcu_Adi,Sertifika_No
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,Yerel_Kuyumcu,703400
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,Tiffany,432931
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,Yerel_Kuyumcu,277350
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,Yerel_Kuyumcu,332504
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,Tiffany,981048


2. Kategorik Verilerin Sayısal Hale Getirilmesi (Encoding)Makine öğrenmesi modelleri "Ideal", "Premium" veya "Cartier" gibi metinleri (String) anlayamaz; sadece sayılarla çalışır. Bu nedenle 2. kod bloğunda şu işlemler yapılmıştır:Dummy Encoding (pd.get_dummies): Veri setindeki kategorik sütunlar (Kesim, Renk vb.) 0 ve 1'lerden oluşan sayısal sütunlara dönüştürülmüştür. Örneğin; "Color" sütunu silinmiş, yerine Color_E, Color_F gibi sütunlar gelmiştir. Eğer elmas E rengindeyse o sütuna 1, değilse 0 yazılır.Kukla Değişken Tuzağı Önlemi (drop_first=True): Kodda kullanılan bu parametre, oluşturulan yeni sütunlardan ilkinin silinmesini sağlar.Mantığı şudur: Eğer bir elmas "Good" veya "Premium" değilse, matematiksel olarak "Fair" (ilk seçenek) olduğu zaten bellidir. Bu fazlalık bilgiyi silmek, modelin kafasının karışmasını (Multicollinearity hatasını) engeller.Veri Ayrımı ($X$ ve $y$):$y$ (Bağımlı Değişken): Tahmin etmeye çalıştığımız price (Fiyat) sütunu ayrılmıştır.$X$ (Bağımsız Değişkenler): Fiyatı etkileyen diğer tüm özellikler (Karat, Boyut, Kesim vb.) ayrı bir matriste toplanmıştır.

In [ ]:
# Kategorik Sütunlar: cut, color, clarity, Kuyumcu_Adi
df_encoded = pd.get_dummies(df, columns=['cut', 'color', 'clarity', 'Kuyumcu_Adi'], drop_first=True)

# Boolean hatası almamak için float yapıyoruz
df_encoded = df_encoded.astype(float)

# Bağımlı (y) ve Bağımsız (X) Değişkenler
y = df_encoded['price']
X = df_encoded.drop('price', axis=1)

print("İşlenmiş Sütunlar:")
print(X.columns.tolist())

İşlenmiş Sütunlar:
['carat', 'depth', 'table', 'x', 'y', 'z', 'Sertifika_No', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'Kuyumcu_Adi_Tiffany', 'Kuyumcu_Adi_Yerel_Kuyumcu']


3.4. Değişken Seçimi (Backward Elimination)
Modelin performansını artırmak amacıyla Geriye Doğru Eleme yöntemi uygulanmış ve OLS analizi sonuçlarına göre veri seti sadeleştirilmiştir:

Anlamsız Değişkenler: Fiyat üzerinde istatistiksel etkisi olmayan (P-değeri > 0.05) 'Sertifika_No' ve bazı 'Kuyumcu_Adi' değişkenleri modelden çıkarılmıştır.

Çoklu Bağlantı (Multicollinearity): Elmasın boyutları (x, y, z) ile ağırlığı (carat) arasında çok yüksek korelasyon tespit edilmiştir. Aynı bilgiyi taşıyan değişkenlerin yarattığı sapmayı önlemek için, boyutları temsilen sadece 'x' tutulmuş; 'y' ve 'z' değişkenleri elenmiştir.

In [ ]:
# OLS Modeli
X_b = np.append(arr=np.ones((len(X), 1)), values=X, axis=1) # Sabit ekle
regressor_OLS = sm.OLS(endog=y, exog=X).fit()

print(regressor_OLS.summary())

print("\n--- ANALİZ ---")
print("1. 'Sertifika_No' ve 'Kuyumcu_Adi' (Tiffany, Yerel vb.) P-değerleri yüksektir -> ELENİYOR.")
print("2. 'y' ve 'z' boyutları 'x' ve 'carat' ile aynı bilgiyi taşıdığı için p-değerleri yüksek çıkabilir -> ELENİYOR.")

# Çöp sütunları ve tekrarlayan boyutları atıyoruz
cols_to_drop = ['Sertifika_No', 'Kuyumcu_Adi_Tiffany', 'Kuyumcu_Adi_Yerel_Kuyumcu']

# Eğer tabloda y ve z'nin P değeri > 0.05 ise onları da ekle (Genelde öyle çıkar)
if 'y' in X.columns: cols_to_drop.append('y')
if 'z' in X.columns: cols_to_drop.append('z')

# Sadece var olanları at (Hata önlemi)
mevcutlar = [c for c in cols_to_drop if c in X.columns]
X_final = X.drop(mevcutlar, axis=1)

print("\nFinal Sütun Listesi:", X_final.columns.tolist())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.959
Model:                            OLS   Adj. R-squared (uncentered):              0.959
Method:                 Least Squares   F-statistic:                          4.888e+04
Date:                Fri, 12 Dec 2025   Prob (F-statistic):                        0.00
Time:                        14:53:40   Log-Likelihood:                     -4.5574e+05
No. Observations:               53940   AIC:                                  9.115e+05
Df Residuals:                   53914   BIC:                                  9.118e+05
Df Model:                          26                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# 1. Model Eğitimi
# Eğitim setini (X_train, y_train) kullanarak modeli eğitiyoruz
model = LinearRegression()
model.fit(X_train, y_train)

# 2. Test Seti Üzerinde Tahmin Yapma
y_pred = model.predict(X_test)

# 3. METRİKLERİN HESAPLANMASI
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# 4. Sonuçları Ekrana Yazdırma
print("-" * 30)
print("MODEL PERFORMANS SONUÇLARI")
print("-" * 30)
print(f"R² Başarısı (Doğruluk): {r2:.4f}")  # 1'e ne kadar yakınsa o kadar iyi
print(f"MAE (Ortalama Hata):    {mae:.2f} $") # Gerçek paradan ne kadar sapıyor?
print(f"MSE (Ortalama Kare Hata): {mse:.2f}") # Hatanın karesi (İstatistiksel puan)
print("-" * 30)

# 5. Modeli Kaydetme
joblib.dump(model, 'elmas_modeli.pkl')
print("Model 'elmas_modeli.pkl' olarak kaydedildi.")

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# 1. Model Eğitimi
# Eğitim setini (X_train, y_train) kullanarak modeli eğitiyoruz
model = LinearRegression()
model.fit(X_train, y_train)

# 2. Test Seti Üzerinde Tahmin Yapma
y_pred = model.predict(X_test)

# 3. METRİKLERİN HESAPLANMASI (Hocanın İstediği 3 Değer)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# 4. Sonuçları Ekrana Yazdırma
print("-" * 30)
print("MODEL PERFORMANS SONUÇLARI")
print("-" * 30)
print(f"R² Başarısı (Doğruluk): {r2:.4f}")  # 1'e ne kadar yakınsa o kadar iyi
print(f"MAE (Ortalama Hata):    {mae:.2f} $") # Gerçek paradan ne kadar sapıyor?
print(f"MSE (Ortalama Kare Hata): {mse:.2f}") # Hatanın karesi (İstatistiksel puan)
print("-" * 30)

# 5. Modeli Kaydetme
joblib.dump(model, 'elmas_modeli.pkl')
print("Model 'elmas_modeli.pkl' olarak kaydedildi.")

------------------------------
MODEL PERFORMANS SONUÇLARI
------------------------------
R² Başarısı (Doğruluk): 0.9189
MAE (Ortalama Hata):    737.21 $
MSE (Ortalama Kare Hata): 1288764.19
------------------------------
Model 'elmas_modeli.pkl' olarak kaydedildi.
